## Periodic boundary conditions

Periodic Green's function for simulation of infinite lattice. Use of symmetry factors for speed-up. Use a different file for each number of periodic boxes and Fourier modes used, in the format of 

    periodic_Nb_Nm.py

In [1]:
from symengine import *
import sympy
import numpy as np
import re
from joblib import Parallel, delayed

Number of periodic boxes and Fourier modes

In [2]:
Nb = 2
Nm = 4

Nbb = 2*Nb +1
N1 = int(-(Nm/2)+1)
N2 = int((Nm/2)+1)

### Definitions and auxiliary functions

In [3]:
xij, yij, zij = symbols("xij, yij, zij", real=True) 
b, eta = symbols("b, eta", real=True, positive=True)

# indices for summation:
## Caution, cannot use lambda, instead use Lambda
## do not use eta as index, it's the viscosity
alpha, beta, gamma, gamma1, gamma2, gamma3, gamma4, kappa, kappa1, kappa2, mu, nu, nu1, nu2, Lambda, eta2, zeta, sigma = symbols(
    'alpha, beta, gamma, gamma1, gamma2, gamma3, gamma4, kappa, kappa1, kappa2, mu, nu, nu1, nu2, Lambda, eta2, zeta, sigma', integer=True)

#new symbols for periodic calculation: boxSize, xi (convergence parameter)
L, xi = symbols("L, xi", real=True, positive=True)
kx, ky, kz = symbols("kx, ky, kz", real=True) 

rij = Matrix([xij, yij, zij])

k0 = 2*pi/L

def norm(r):
    result=0
    for i in range(3):
        result += r[i]**2
    return sqrt(result)

KroneckerDelta = eye(3)

def eps(alpha, beta, gamma):
    return LeviCivita(alpha,beta,gamma)

### Green's function

In [4]:
##pass xi*norm(r) and norm(k) for A, B and C

def A(x): ##are these correct? Don't match Pozrikidis
    return erfc(x) + 2/sqrt(pi)*(2*x**3 - 3*x)*exp(-x**2)

def B(x):
    return erfc(x) + 2/sqrt(pi)*(x - 2*x**3)*exp(-x**2)

def C(k):
    return (1 + 1/4*k**2/xi**2 + 1/8*k**4/xi**4)*exp(-1/4*k**4/xi**4)

def M1(xij, yij, zij, alpha,beta): ##factors of 8*pi?
    rij = Matrix([xij, yij, zij])
    return A(xi*norm(rij))*KroneckerDelta[alpha,beta]/norm(rij) + B(xi*norm(rij))*rij[alpha]*rij[beta]/norm(rij)**3

def M2(xij, yij, zij, kx, ky, kz, alpha,beta): ## k*rij is dot product
    rij = Matrix([xij, yij, zij])
    k = Matrix([kx, ky, kz]) 
    kDOTrij = xij*kx + yij*ky + zij*kz
    return 1/(eta)*C(norm(k))*(KroneckerDelta[alpha,beta]/norm(k)**2 + k[alpha]*k[beta]/norm(k)**4)*cos(kDOTrij) ##take the real part (?) follow Pozrikidis (3.14)
                                                                                                                ## just use cos instead of exp(I*kDOTrij)

##define function for periodic Green's function, evaluate it once and save it as another function, as to not have to evaluate it every time 
def GreenReal(alpha, beta): ##eventually evaluate xij=L
    green = 0

    for ii in range(Nbb):
        dx = (ii - Nb)*xij
        for jj in range(Nbb):
            dy = (jj - Nb)*yij
            for kk in range(Nbb):
                dz = (kk - Nb)*zij
                
                if ii==jj==kk: ##self-interaction added later by hand
                    pass
                else:
                    green += M1(dx, dy, dz, alpha, beta)
                    
    return green


def GreenFourier(alpha,beta): ##eventually evaluate xij=0 and take real part
    green = 0 
    
    for ii in range(N1, N2): 
        kx = k0*ii
        for jj in range(N1,N2):
            ky = k0*jj
            for kk in range(N1, N2):
                kz = k0*kk
                if kx != 0 or ky != 0 or kz != 0:
                    green += M2(xij, yij, zij, kx, ky, kz, alpha, beta)

    return green

In [5]:
sympy.simplify(GreenReal(0,1).subs({xij:L, yij:L, zij:L}))

8*L**2*xi**3*exp(-3*L**2*xi**2)/sqrt(pi) + 32*L**2*xi**3*exp(-12*L**2*xi**2)/sqrt(pi) - 4*xi*exp(-3*L**2*xi**2)/(3*sqrt(pi)) - 4*xi*exp(-12*L**2*xi**2)/(3*sqrt(pi)) - 2*sqrt(3)*(L**2)**(3/2)*erfc(sqrt(3)*xi*sqrt(L**2))/(9*L**4) - sqrt(3)*(L**2)**(3/2)*erfc(2*sqrt(3)*xi*sqrt(L**2))/(9*L**4)

In [6]:
sympy.simplify(GreenFourier(0,1).subs({xij:0, yij:0, zij:0}))


((36*L**4*xi**4 + 432.0*pi**2*L**2*xi**2 + 10368.0*pi**4)*exp(580.0*pi**4/(L**4*xi**4)) + (81*L**4*xi**4 + 648.0*pi**2*L**2*xi**2 + 10368.0*pi**4)*exp(900.0*pi**4/(L**4*xi**4)) + (128*L**4*xi**4 + 1152.0*pi**2*L**2*xi**2 + 20736.0*pi**4)*exp(832.0*pi**4/(L**4*xi**4)))*exp(-1156.0*pi**4/(L**4*xi**4))/(5184*pi**2*L**2*eta*xi**4)

In [7]:
## First compute Green's functions and save in matrix & then in function, to not have to compute it every time (we fix the number of modes)
## then change below matrix elements computations to include an argument G, meaning which Green's function is used
##compute all matrix elements for both and then simply sum them GP = sum M1 + 1/V sum M2 - simplify too slow

In [8]:
zeros(3,3)

[0, 0, 0]
[0, 0, 0]
[0, 0, 0]

In [9]:
matGR = zeros(3,3)
for i in range(3):
    for j in range(3):
        matGR[i,j] = GreenReal(i,j)
        
matGF = zeros(3,3)
for i in range(3):
    for j in range(3):
        matGF[i,j] = GreenFourier(i,j)
        
def GR(alpha, beta):
    return matGR[alpha,beta]

def GF(alpha,beta):
    return matGF[alpha,beta]

### Calculus auxiliary functions

In [10]:
def Laplacian(G):
    return diff(G, rij[0], rij[0]) + diff(G, rij[1], rij[1]) + diff(G, rij[2], rij[2])

def Curl(G, alpha, beta): ## the function to take the curl of and the two remaining indices 
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(alpha,i,j)*diff(G(j,beta),rij[i])
    return result

def tilde_Curl(G, alpha, beta): ##introduce for convenience, minus sign already taken into account in matrix elements
    result=0
    for i in range(3):
        for j in range(3):
            result += eps(beta,i,j)*diff(G(alpha, j),rij[i])
    return result

def CurlCurl(G, alpha, beta): ## the function to take the curl of and the two remaining indices 
    result=0
    for i in range(3):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    result += eps(alpha,i,j)*eps(beta,k,l)*diff(G(j,l), rij[i], rij[k])
    return result


def SumG(G, sigma, eta2, mu):
    result=0
    for i in range(3):
        result += eps(sigma, eta2, i)*Laplacian( G(mu, i) )
    return result


def SumG3s1Sym(G, mu, nu2, alpha, gamma1, gamma2):
    result=0
    for i in range(3):
        result += eps(mu,nu2,i)*g3s1Sym(G, alpha,gamma1,gamma2,i) ##add argument G
    return result

In [11]:
##auxiliary functions
def delG(G, alpha, beta, gamma):
    return diff( G(alpha,beta), rij[gamma])

def lapG(G, alpha,beta):
    return Laplacian( G(alpha,beta) )

def dellapG(G, alpha, beta, kappa1):
    return diff(lapG(G, alpha, beta), rij[kappa1])

def deldelG(G, zeta, beta, kappa1, nu):
    return diff(delG(G, zeta, beta, kappa1), rij[nu])

def deldellapG(G, alpha, beta, gamma1, kappa1):
    return diff(dellapG(G, alpha,beta, gamma1), rij[kappa1])

def deldeldelG(G, alpha, beta, gamma1, gamma2, gamma3):
    return diff(deldelG(G, alpha, beta, gamma1, gamma2), rij[gamma3])

def deldeldeldelG(G, alpha, beta, gamma1, gamma2, gamma3, gamma4):
    return diff(deldeldelG(G, alpha, beta, gamma1, gamma2, gamma3), rij[gamma4])

In [12]:
# GLL  ##simplify later, when filling matrix elements, after subsituting for xij -- much faster
def G1s1s(G, alpha, beta):
    return G(alpha, beta) + b**2/3*lapG(G, alpha, beta)

def G1s2a(G, alpha, beta):
    return -1/2*b*tilde_Curl(G, alpha,beta)

def G2a1s(G, alpha, beta):
    return b*Curl(G, alpha,beta)

def G2a2a(G, alpha, beta):
    return -1/2*b*b*CurlCurl(G, alpha,beta)

In [13]:
## GL2s and G2sL
def G1s2s(G, alpha, kappa1, beta):
    return -1/2*b*((delG(G, alpha, beta, kappa1) + delG(G, alpha, kappa1, beta)) + 4*b**2/15*(dellapG(G, alpha, beta, kappa1) + dellapG(G, alpha, kappa1, beta)))

def G2a2s(G, alpha, kappa1, beta):
    return -1/2*b*b*(diff(Curl(G, alpha, beta),rij[kappa1]) + diff(Curl(G, alpha, kappa1),rij[beta]))

def G2s1s(G, alpha, gamma1, beta):
    return 1/2*b*((delG(G, alpha, beta, gamma1) + delG(G, gamma1, beta, alpha)) + 4*b**2/15*(dellapG(G, alpha,beta,gamma1) + dellapG(G, gamma1, beta, alpha)))

def G2s2a(G, alpha, gamma1, mu):
    return -1/4*b*b*(diff(tilde_Curl(G, gamma1,mu), rij[alpha]) + diff(tilde_Curl(G, alpha,mu), rij[gamma1])) #sign change from -> eps(mu, kappa1, beta)


## G2s2s
def G2s2s(G, alpha, gamma1, kappa1, beta):
    return -1/4*b*b*((deldelG(G, alpha, beta, gamma1, kappa1) + deldelG(G, gamma1, beta, alpha, kappa1)) + 
                      (deldelG(G, alpha,kappa1, gamma1, beta) + deldelG(G, gamma1, kappa1, alpha, beta)) +
                      b**2/5*(deldellapG(G, alpha,beta,gamma1,kappa1) + deldellapG(G, gamma1,beta,alpha,kappa1)) + 
                      b**2/5*(deldellapG(G, alpha, kappa1, gamma1, beta) + deldellapG(G, gamma1,kappa1,alpha,beta)))


## higher matrix elements
def G1s3t(G, alpha, beta):
    return -1/5*b**2*lapG(G, alpha,beta)


# symmetry for matrix elements of form l sigma, 3s
def g13sSym(G, alpha,kappa2,kappa1,beta):
    return 1/3*(deldelG(G, alpha,beta,kappa1,kappa2) + deldelG(G, alpha,kappa2,kappa1,beta) + deldelG(G, alpha,kappa1,beta,kappa2)) - 1/15*(
        KroneckerDelta[kappa1,kappa2]*lapG(G, alpha,beta) + KroneckerDelta[kappa1,beta]*lapG(G, alpha,kappa2) + KroneckerDelta[beta,kappa2]*lapG(G, alpha,kappa1))

def G1s3s(G, alpha,kappa2, kappa1, beta):
    return b**2*(g13sSym(G, alpha,kappa2,kappa1,beta) + 5*b**2/21*Laplacian(g13sSym(G, alpha,kappa2,kappa1,beta)))


# symmetry of matrix elements of form 3s, l sigma
def g3s1Sym(G, alpha,gamma1,gamma2,beta):
    return 1/3*(deldelG(G, alpha,beta,gamma1,gamma2) + deldelG(G, gamma1,beta,gamma2,alpha) + deldelG(G, gamma2,beta,alpha,gamma1)) - 1/15*(
        KroneckerDelta[gamma1, gamma2]*lapG(G, alpha,beta) + KroneckerDelta[gamma2,alpha]*lapG(G, gamma1,beta) + KroneckerDelta[alpha,gamma1]*lapG(G, gamma2,beta))

def G3s1s(G, alpha,gamma1,gamma2,beta):
    return b**2*(g3s1Sym(G, alpha,gamma1,gamma2,beta) + 5*b**2/21*Laplacian(g3s1Sym(G, alpha,gamma1,gamma2,beta)))

def G3a1s(G, Lambda, kappa2, beta):
    return 1/2*b**2*(diff(Curl(G, Lambda, beta), rij[kappa2]) + diff(Curl(G, kappa2,beta), rij[Lambda]))

def G1s3a(G, alpha, nu2, sigma):
    return 2/3*b**2*(diff(tilde_Curl(G, alpha,nu2),rij[sigma]) - 1/3*SumG(G, nu2,sigma,alpha)) # sign change due to --> eps(sigma,nu1,beta)

def g3sCurlSym(G, alpha,gamma1,gamma2,mu):
    return 1/3*(diff(tilde_Curl(G, alpha,mu), rij[gamma1],rij[gamma2]) + diff(tilde_Curl(G, gamma1,mu), rij[gamma2],rij[alpha]) + diff(tilde_Curl(G, gamma2,mu), rij[alpha],rij[gamma1])) 
        

def G3s2a(G, alpha,gamma1,gamma2,mu):
    return -1/2*b**3*g3sCurlSym(G, alpha, gamma1, gamma2, mu)## sign change due to -> eps(mu, kappa1, beta)

def G3a2a(G, Lambda, kappa2, mu):
    return -1/4*b**3*(diff(CurlCurl(G, Lambda,mu), rij[kappa2]) + diff(CurlCurl(G, kappa2,mu), rij[Lambda])) #sign change -> eps(mu,kappa1,beta)

def G2s3t(G, alpha,gamma1,nu2):
    return -b**3/10*(dellapG(G, alpha,nu2,gamma1) + dellapG(G, gamma1,nu2,alpha))

def G3s3t(G, alpha,gamma1,gamma2,nu2):
    return -b**4/5*Laplacian(g3s1Sym(G, alpha,gamma1,gamma2,nu2))


def G2s3a(G, alpha,gamma1,mu,nu2): #sign change -> eps(mu,nu1,beta)
    return b**3/3*( diff(tilde_Curl(G, alpha,mu), rij[gamma1], rij[nu2]) - 1/3*diff(SumG(G, mu,nu2,alpha), rij[gamma1])
                   + diff(tilde_Curl(G, gamma1,mu), rij[alpha], rij[nu2]) - 1/3*diff(SumG(G, mu,nu2,gamma1), rij[alpha]))

def G3a3a(G, Lambda, kappa2, mu, nu2): #sign change -> eps(mu,nu1,beta)
    return b**4/3*(diff(CurlCurl(G, Lambda,mu), rij[nu2], rij[kappa2]) + diff(CurlCurl(G, kappa2,mu), rij[nu2], rij[Lambda]))


def G3s3a(G, alpha,gamma1,gamma2,mu,nu2): #sign change -> eps(mu,nu1,beta)
    return 2*b**4/3*(diff(g3sCurlSym(G, alpha,gamma1,gamma2,mu), rij[nu2]) - 1/3*Laplacian(SumG3s1Sym(G, mu,nu2,alpha,gamma1,gamma2)))


def g2s3sSym(G, alpha,gamma1,kappa2, kappa1, beta):
    return diff(g13sSym(G, alpha,kappa2,kappa1,beta), rij[gamma1]) + diff(g13sSym(G, gamma1, kappa2, kappa1, beta), rij[alpha])

def G2s3s(G, alpha,gamma1,kappa2, kappa1, beta):
    return 1/2*b**3*(g2s3sSym(G, alpha,gamma1,kappa2, kappa1, beta) + 6*b**2/35*Laplacian(g2s3sSym(G, alpha,gamma1,kappa2, kappa1, beta)))


def gCurl3sSym(G, tau, kappa2, kappa1, beta):
    return 1/3*(diff(Curl(G, tau, beta), rij[kappa1],rij[kappa2]) + diff(Curl(G, tau,kappa1), rij[kappa2],rij[beta]) + diff(Curl(G, tau,kappa2), rij[beta],rij[kappa1])) 


def G3a3s(G, Lambda, eta2, kappa2, kappa1, beta):
    return 1/2*b**4*(diff(gCurl3sSym(G, Lambda, kappa1, kappa2, beta), rij[eta2]) + diff(gCurl3sSym(G, eta2, kappa1, kappa2, beta), rij[Lambda]))


def deldelg13sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return diff(g13sSym(G, alpha,kappa2,kappa1,beta), rij[gamma1], rij[gamma2])

def lapg13sSym(G, alpha,kappa2,kappa1,beta):
    return Laplacian(g13sSym(G, alpha,kappa2,kappa1,beta))

def g3s3sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return 1/3*(deldelg13sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta) + deldelg13sSym(G, gamma1,gamma2,alpha,kappa2,kappa1,beta) + 
                deldelg13sSym(G, gamma2,alpha,gamma1,kappa2,kappa1,beta)) - 1/15*(
                KroneckerDelta[gamma1,gamma2]*lapg13sSym(G, alpha, kappa2, kappa1, beta) + 
                KroneckerDelta[gamma2,alpha]*lapg13sSym(G, gamma1, kappa2, kappa1, beta) + 
                KroneckerDelta[alpha,gamma1]*lapg13sSym(G, gamma2, kappa2, kappa1, beta))

def G3s3s(G, alpha,gamma1,gamma2,kappa2,kappa1,beta):
    return b**4*(g3s3sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta) + b**2/7*Laplacian(g3s3sSym(G, alpha,gamma1,gamma2,kappa2,kappa1,beta)))


def G3a2s(G, Lambda, kappa2, kappa1, beta):
    return -1/4*b**3*(diff(Curl(G, Lambda,beta), rij[kappa1], rij[kappa2]) + diff(Curl(G, Lambda,kappa1), rij[beta], rij[kappa2])
                       + diff(Curl(G, kappa2,beta), rij[kappa1], rij[Lambda]) + diff(Curl(G, kappa2,kappa1), rij[beta], rij[Lambda]))


def g3s2sSym(G, alpha,gamma1,gamma2, kappa1, beta):
    return diff(g3s1Sym(G, alpha,gamma1,gamma2,beta), rij[kappa1]) + diff(g3s1Sym(G, alpha,gamma1,gamma2,kappa1), rij[beta])

def G3s2s(G, alpha,gamma1,gamma2,kappa1,beta):
    return -1/2*b**3*(g3s2sSym(G, alpha,gamma1,gamma2, kappa1, beta) + 6*b**2/35*Laplacian(g3s2sSym(G, alpha,gamma1,gamma2, kappa1, beta)))


def G2a3a(G, Lambda, mu, nu2): # sign change -> eps(mu,nu1,beta)
    return 2*b**3/3*diff(CurlCurl(G, Lambda, mu), rij[nu2])


def G2a3s(G, Lambda,kappa2,kappa1,beta):
    return b**3*gCurl3sSym(G, Lambda, kappa1, kappa2, beta)

In [14]:
## zero elements

def G3t1s(G, alpha,beta):
    return 0

def G3t2a(G, alpha,beta):
    return 0

def G3t2s(G, alpha, beta, kappa1):
    return 0

def G3t3t(G, alpha,beta):
    return 0

def G3t3a(G, alpha, beta, kappa1):
    return 0

def G3t3s(G, alpha, beta, kappa1, kappa2):
    return 0

def G2a3t(G, alpha,beta):
    return 0

def G3a3t(G, alpha, gamma1, beta):
    return 0

## Matrix assembly

In [15]:
n_jobs = 1  ## symEngine does not parallelise properly

In [16]:
modes = ['1s', '2a', '2s', '3t', '3a', '3s']
modesV = ['2s', '3t'] ##possible slip modes, for double-layer matrix elements
SLDL  = ['G']#, 'K']
dim   = {'1s':3, '2a':3, '2s':5, '3t':3, '3a':5, '3s':7}

### 1) Fourier part of Green's function

In [17]:
%%time
sldl = 'G' # single-layer
GRF = 'GF' # Fourier space part of Green's function
rpl = 0    # what are xij, yij, zij replaced with? rpl = replacement
for mode1 in modes:
    for mode2 in modes:
        if mode1 in ['1s', '2a']:
            if mode2 in ['1s', '2a', '3t']:
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, kj) for ki in range(3) 
                                                                                    for kj in range(3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')

            elif mode2 in ['2s', '3a']:                ## GLH needs symmetry factors
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, kj, lj) for ki in range(3)
                                                                                        for kj in range(2)
                                                                                            for lj in range(kj,3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')

            elif mode2 =='3s':
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, kj, lj, mj) for ki in range(3)
                                                                                            for kj in range(2)
                                                                                                for lj in range(kj,2)
                                                                                                    for mj in range(lj,3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')



        elif mode1 == '3t':
            if mode2 in ['1s', '2a', '3t']:
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, kj) for ki in range(3) 
                                                                                    for kj in range(3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')

            elif mode2 in ['2s', '3a']:
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, kj, lj) for ki in range(3)
                                                                                        for kj in range(2)
                                                                                            for lj in range(kj,3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')

            elif mode2 =='3s':
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, kj, lj, mj) for ki in range(3)
                                                                                            for kj in range(2)
                                                                                                for lj in range(kj,2)
                                                                                                    for mj in range(lj,3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')



        elif mode1 in ['2s', '3a']:
            if mode2 in ['1s', '2a', '3t']:
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, li, kj) for ki in range(2) 
                                                                                        for li in range(ki,3)
                                                                                            for kj in range(3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')

            elif mode2 in ['2s', '3a']:
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, li, kj, lj) for ki in range(2)
                                                                                            for li in range(ki,3)
                                                                                                for kj in range(2)
                                                                                                    for lj in range(kj,3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')

            elif mode2 =='3s':
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, li, kj, lj, mj) for ki in range(2)
                                                                                                for li in range(ki,3)
                                                                                                    for kj in range(2)
                                                                                                        for lj in range(kj,2)
                                                                                                            for mj in range(lj,3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')



        elif mode1 == '3s':
            if mode2 in ['1s', '2a', '3t']:
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, li, mi, kj) for ki in range(2) 
                                                                                            for li in range(ki,2)
                                                                                                for mi in range(li,3)
                                                                                                    for kj in range(3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')

            elif mode2 in ['2s', '3a']:
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, li, mi, kj, lj) for ki in range(2)
                                                                                                for li in range(ki,2)
                                                                                                    for mi in range(li,3)
                                                                                                        for kj in range(2)
                                                                                                            for lj in range(kj,3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')

            elif mode2 =='3s':
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, li, mi, kj, lj, mj) for ki in range(2)
                                                                                                    for li in range(ki,2)
                                                                                                        for mi in range(li,3)
                                                                                                            for kj in range(2)
                                                                                                                for lj in range(kj,2)
                                                                                                                    for mj in range(lj,3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')

G1s1s done
G1s2a done
G1s2s done
G1s3t done
G1s3a done
G1s3s done
G2a1s done
G2a2a done
G2a2s done
G2a3t done
G2a3a done
G2a3s done
G2s1s done
G2s2a done
G2s2s done
G2s3t done
G2s3a done
G2s3s done
G3t1s done
G3t2a done
G3t2s done
G3t3t done
G3t3a done
G3t3s done
G3a1s done
G3a2a done
G3a2s done
G3a3t done
G3a3a done
G3a3s done
G3s1s done
G3s2a done
G3s2s done
G3s3t done
G3s3a done
G3s3s done
CPU times: user 20.3 s, sys: 32.9 ms, total: 20.4 s
Wall time: 20.3 s


### 2) real Green's function (takes longer)

In [18]:
%%time
sldl = 'G' # single-layer
GRF = 'GR' # real part of Green's function
rpl = L    # what are xij, yij, zij replaced with? rpl = replacement
for mode1 in modes:
    for mode2 in modes:
        if mode1 in ['1s', '2a']:
            if mode2 in ['1s', '2a', '3t']:
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, kj) for ki in range(3) 
                                                                                    for kj in range(3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')

            elif mode2 in ['2s', '3a']:                ## GLH needs symmetry factors
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, kj, lj) for ki in range(3)
                                                                                        for kj in range(2)
                                                                                            for lj in range(kj,3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')

            elif mode2 =='3s':
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, kj, lj, mj) for ki in range(3)
                                                                                            for kj in range(2)
                                                                                                for lj in range(kj,2)
                                                                                                    for mj in range(lj,3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')



        elif mode1 == '3t':
            if mode2 in ['1s', '2a', '3t']:
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, kj) for ki in range(3) 
                                                                                    for kj in range(3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')

            elif mode2 in ['2s', '3a']:
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, kj, lj) for ki in range(3)
                                                                                        for kj in range(2)
                                                                                            for lj in range(kj,3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')

            elif mode2 =='3s':
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, kj, lj, mj) for ki in range(3)
                                                                                            for kj in range(2)
                                                                                                for lj in range(kj,2)
                                                                                                    for mj in range(lj,3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')



        elif mode1 in ['2s', '3a']:
            if mode2 in ['1s', '2a', '3t']:
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, li, kj) for ki in range(2) 
                                                                                        for li in range(ki,3)
                                                                                            for kj in range(3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')

            elif mode2 in ['2s', '3a']:
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, li, kj, lj) for ki in range(2)
                                                                                            for li in range(ki,3)
                                                                                                for kj in range(2)
                                                                                                    for lj in range(kj,3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')

            elif mode2 =='3s':
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, li, kj, lj, mj) for ki in range(2)
                                                                                                for li in range(ki,3)
                                                                                                    for kj in range(2)
                                                                                                        for lj in range(kj,2)
                                                                                                            for mj in range(lj,3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')



        elif mode1 == '3s':
            if mode2 in ['1s', '2a', '3t']:
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, li, mi, kj) for ki in range(2) 
                                                                                            for li in range(ki,2)
                                                                                                for mi in range(li,3)
                                                                                                    for kj in range(3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')

            elif mode2 in ['2s', '3a']:
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, li, mi, kj, lj) for ki in range(2)
                                                                                                for li in range(ki,2)
                                                                                                    for mi in range(li,3)
                                                                                                        for kj in range(2)
                                                                                                            for lj in range(kj,3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')

            elif mode2 =='3s':
                globals()['list'+GRF+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(globals()[sldl+mode1+mode2])(globals()[GRF], ki, li, mi, kj, lj, mj) for ki in range(2)
                                                                                                    for li in range(ki,2)
                                                                                                        for mi in range(li,3)
                                                                                                            for kj in range(2)
                                                                                                                for lj in range(kj,2)
                                                                                                                    for mj in range(lj,3))
                globals()['mat'+GRF+mode1+mode2] = Matrix(globals()['list'+GRF+mode1+mode2]).subs({xij:rpl, yij:rpl, zij:rpl}).reshape(dim[mode1],dim[mode2])
                print(sldl+mode1+mode2+' done')

G1s1s done
G1s2a done
G1s2s done
G1s3t done
G1s3a done
G1s3s done
G2a1s done
G2a2a done
G2a2s done
G2a3t done
G2a3a done
G2a3s done
G2s1s done
G2s2a done
G2s2s done
G2s3t done
G2s3a done
G2s3s done
G3t1s done
G3t2a done
G3t2s done
G3t3t done
G3t3a done
G3t3s done
G3a1s done
G3a2a done
G3a2s done
G3a3t done
G3a3a done
G3a3s done
G3s1s done
G3s2a done
G3s2s done
G3s3t done
G3s3a done
G3s3s done
CPU times: user 2min, sys: 1.33 s, total: 2min 1s
Wall time: 2min 1s


### 3) add Green's functions

In [19]:
for mode1 in modes:
    for mode2 in modes:
        globals()['mat'+sldl+mode1+mode2] = 1/(8*pi*eta) * globals()['matGR'+mode1+mode2] + 1/L**3 * globals()['matGF'+mode1+mode2]
        print(sldl+mode1+mode2+' done')

G1s1s done
G1s2a done
G1s2s done
G1s3t done
G1s3a done
G1s3s done
G2a1s done
G2a2a done
G2a2s done
G2a3t done
G2a3a done
G2a3s done
G2s1s done
G2s2a done
G2s2s done
G2s3t done
G2s3a done
G2s3s done
G3t1s done
G3t2a done
G3t2s done
G3t3t done
G3t3a done
G3t3s done
G3a1s done
G3a2a done
G3a2s done
G3a3t done
G3a3a done
G3a3s done
G3s1s done
G3s2a done
G3s2s done
G3s3t done
G3s3a done
G3s3s done


### Create double-layer matrices

In [20]:
for mode1 in modes:
    for mode2 in modesV:
        if mode2 == '2s':
            globals()['matK'+mode1+mode2] = 8*pi*eta*b/3 * globals()['matG'+mode1+mode2]
            print('K'+mode1+mode2+' done')
            
        elif mode2 == '3t':
            globals()['matK'+mode1+mode2] = 6*pi*eta*b/5 * globals()['matG'+mode1+mode2]
            print('K'+mode1+mode2+' done')

K1s2s done
K1s3t done
K2a2s done
K2a3t done
K2s2s done
K2s3t done
K3t2s done
K3t3t done
K3a2s done
K3a3t done
K3s2s done
K3s3t done


In [23]:
## save all G's and K's to files, and re-load into a different sympy file for simplification over parallel modes ? Nope

In [22]:
#text_file = open("matrixElements/G3s3s.txt", "w")
#nn = text_file.write(str(matG3s3s))
#text_file.close()
#nn ##that's a 70MB file lol

In [24]:
matGF1s1s[0,1]

(1/3)*b**2*((-1/3)*exp(pi**4*-576.0/(L**4*xi**4))*(1 + 288.0*pi**4/(L**4*xi**4) + 12.0*pi**2/(L**2*xi**2))/eta + (-8/9)*exp(pi**4*-324.0/(L**4*xi**4))*(1 + 162.0*pi**4/(L**4*xi**4) + 9.0*pi**2/(L**2*xi**2))/eta + (-1/2)*exp(pi**4*-256.0/(L**4*xi**4))*(1 + 128.0*pi**4/(L**4*xi**4) + 8.0*pi**2/(L**2*xi**2))/eta) + (1/144)*L**2*exp(pi**4*-576.0/(L**4*xi**4))*(1 + 288.0*pi**4/(L**4*xi**4) + 12.0*pi**2/(L**2*xi**2))/(pi**2*eta) + (2/81)*L**2*exp(pi**4*-324.0/(L**4*xi**4))*(1 + 162.0*pi**4/(L**4*xi**4) + 9.0*pi**2/(L**2*xi**2))/(pi**2*eta) + (1/64)*L**2*exp(pi**4*-256.0/(L**4*xi**4))*(1 + 128.0*pi**4/(L**4*xi**4) + 8.0*pi**2/(L**2*xi**2))/(pi**2*eta)

In [47]:
arguments = [b, eta, L, xi]
f = lambdify(arguments, [matGF1s1s[0,0]])

In [48]:
f(1., 0.1, 1, 1)

array(-3.07560528e-166)

In [21]:
matGF1s1s[0,1]

L**2*(1 + 8.0*pi**2/(L**2*xi**2) + 128.0*pi**4/(L**4*xi**4))*exp(-256.0*pi**4/(L**4*xi**4))/(64*pi**2*eta) + 2*L**2*(1 + 9.0*pi**2/(L**2*xi**2) + 162.0*pi**4/(L**4*xi**4))*exp(-324.0*pi**4/(L**4*xi**4))/(81*pi**2*eta) + L**2*(1 + 12.0*pi**2/(L**2*xi**2) + 288.0*pi**4/(L**4*xi**4))*exp(-576.0*pi**4/(L**4*xi**4))/(144*pi**2*eta) + b**2*(-(1 + 8.0*pi**2/(L**2*xi**2) + 128.0*pi**4/(L**4*xi**4))*exp(-256.0*pi**4/(L**4*xi**4))/(2*eta) - 8*(1 + 9.0*pi**2/(L**2*xi**2) + 162.0*pi**4/(L**4*xi**4))*exp(-324.0*pi**4/(L**4*xi**4))/(9*eta) - (1 + 12.0*pi**2/(L**2*xi**2) + 288.0*pi**4/(L**4*xi**4))*exp(-576.0*pi**4/(L**4*xi**4))/(3*eta))/3

In [32]:
%%time
sympy.simplify(matGR1s1s[0,0])

CPU times: user 14.7 s, sys: 12.1 ms, total: 14.7 s
Wall time: 14.8 s


8*L**206*erfc(xi*sqrt(L**2))/(L**2)**(207/2) + 4*L**206*erfc(2*xi*sqrt(L**2))/(L**2)**(207/2) + 32*L**206*erfc(3*xi*sqrt(L**2))/(3*(L**2)**(207/2)) + 8*sqrt(2)*L**206*erfc(sqrt(2)*xi*sqrt(L**2))/(L**2)**(207/2) + 4*sqrt(2)*L**206*erfc(2*sqrt(2)*xi*sqrt(L**2))/(L**2)**(207/2) + 8*sqrt(3)*L**206*erfc(sqrt(3)*xi*sqrt(L**2))/(3*(L**2)**(207/2)) + 4*sqrt(3)*L**206*erfc(2*sqrt(3)*xi*sqrt(L**2))/(3*(L**2)**(207/2)) + 32*sqrt(5)*L**206*erfc(sqrt(5)*xi*sqrt(L**2))/(5*(L**2)**(207/2)) + 16*sqrt(6)*L**206*erfc(sqrt(6)*xi*sqrt(L**2))/(3*(L**2)**(207/2)) + 16*L**204*b**2*erfc(xi*sqrt(L**2))/(3*(L**2)**(207/2)) + 8*L**204*b**2*erfc(2*xi*sqrt(L**2))/(3*(L**2)**(207/2)) + 64*L**204*b**2*erfc(3*xi*sqrt(L**2))/(81*(L**2)**(207/2)) + 4*sqrt(2)*L**204*b**2*erfc(sqrt(2)*xi*sqrt(L**2))/(3*(L**2)**(207/2)) + 2*sqrt(2)*L**204*b**2*erfc(2*sqrt(2)*xi*sqrt(L**2))/(3*(L**2)**(207/2)) + 832*sqrt(5)*L**204*b**2*erfc(sqrt(5)*xi*sqrt(L**2))/(375*(L**2)**(207/2)) + 8*sqrt(6)*L**204*b**2*erfc(sqrt(6)*xi*sqrt(L**2))/(9*

In [25]:
sympy.simplify(expand(matGF1s1s[0,1]))

0.00694444444444444*L**2*exp(-576.0*pi**4/(L**4*xi**4))/(pi**2*eta) + 0.0246913580246914*L**2*exp(-324.0*pi**4/(L**4*xi**4))/(pi**2*eta) + 0.015625*L**2*exp(-256.0*pi**4/(L**4*xi**4))/(pi**2*eta) - 0.111111111111111*b**2*exp(-576.0*pi**4/(L**4*xi**4))/eta - 0.296296296296296*b**2*exp(-324.0*pi**4/(L**4*xi**4))/eta - 0.166666666666667*b**2*exp(-256.0*pi**4/(L**4*xi**4))/eta + 0.0833333333333333*exp(-576.0*pi**4/(L**4*xi**4))/(eta*xi**2) + 0.222222222222222*exp(-324.0*pi**4/(L**4*xi**4))/(eta*xi**2) + 0.125*exp(-256.0*pi**4/(L**4*xi**4))/(eta*xi**2) - 1.33333333333333*pi**2*b**2*exp(-576.0*pi**4/(L**4*xi**4))/(L**2*eta*xi**2) - 2.66666666666667*pi**2*b**2*exp(-324.0*pi**4/(L**4*xi**4))/(L**2*eta*xi**2) - 1.33333333333333*pi**2*b**2*exp(-256.0*pi**4/(L**4*xi**4))/(L**2*eta*xi**2) + 2.0*pi**2*exp(-576.0*pi**4/(L**4*xi**4))/(L**2*eta*xi**4) + 4.0*pi**2*exp(-324.0*pi**4/(L**4*xi**4))/(L**2*eta*xi**4) + 2.0*pi**2*exp(-256.0*pi**4/(L**4*xi**4))/(L**2*eta*xi**4) - 32.0*pi**4*b**2*exp(-576.0*pi*

_________
__________


### Does not work:

In [27]:
def SimplifyG(matG, alpha,beta):
    return sympy.simplify( matG(alpha,beta) )

In [ ]:
n_jobs = -1 ## does not work with symengine, and cannot seem to convert to sympy - try without simplifying for now - how large will file be? 
listGF1s1s = Parallel(n_jobs=n_jobs)(delayed(SimplifyG)(matGF1s1s, i, j ) for i in range(3) for j in range(3))

In [ ]:
%%time
sldl = 'G'
for mode1 in modes:
    for mode2 in modes:
        if mode1 in ['1s', '2a']:
            if mode2 in ['1s', '2a', '3t']:
                globals()['list'+sldl+mode1+mode2] = Parallel(n_jobs=n_jobs)(
                    delayed(SimplifyG)(globals()['mat'+sldl+mode1+mode2], ki, kj) for ki in range(3) 
                                                                                        for kj in range(3))
                globals()['array'+sldl+mode1+mode2] = np.asarray(globals()['list'+sldl+mode1+mode2]).reshape(dim[mode1],dim[mode2])
                globals()['mat'+sldl+mode1+mode2] = sympy.Matrix(globals()['array'+sldl+mode1+mode2])
                print(sldl+mode1+mode2+' done')

In [24]:
Simplif

'G'

In [19]:
%%time
sympy.simplify(matG1s1s[0,0]) ##simplify in parallel?

CPU times: user 26.4 s, sys: 27.7 ms, total: 26.4 s
Wall time: 26.5 s


8*L**4*b**2*xi**7*exp(-L**2*xi**2)/(3*pi**(3/2)*eta) + 64*L**4*b**2*xi**7*exp(-2*L**2*xi**2)/(3*pi**(3/2)*eta) + 24*L**4*b**2*xi**7*exp(-3*L**2*xi**2)/(pi**(3/2)*eta) + 512*L**4*b**2*xi**7*exp(-4*L**2*xi**2)/(3*pi**(3/2)*eta) + 2720*L**4*b**2*xi**7*exp(-5*L**2*xi**2)/(3*pi**(3/2)*eta) + 1152*L**4*b**2*xi**7*exp(-6*L**2*xi**2)/(pi**(3/2)*eta) + 4096*L**4*b**2*xi**7*exp(-8*L**2*xi**2)/(3*pi**(3/2)*eta) + 3168*L**4*b**2*xi**7*exp(-9*L**2*xi**2)/(pi**(3/2)*eta) + 1536*L**4*b**2*xi**7*exp(-12*L**2*xi**2)/(pi**(3/2)*eta) + L**4*erfc(xi*sqrt(L**2))/(pi*eta*(L**2)**(5/2)) + L**4*erfc(2*xi*sqrt(L**2))/(2*pi*eta*(L**2)**(5/2)) + 4*L**4*erfc(3*xi*sqrt(L**2))/(3*pi*eta*(L**2)**(5/2)) + sqrt(2)*L**4*erfc(sqrt(2)*xi*sqrt(L**2))/(pi*eta*(L**2)**(5/2)) + sqrt(2)*L**4*erfc(2*sqrt(2)*xi*sqrt(L**2))/(2*pi*eta*(L**2)**(5/2)) + sqrt(3)*L**4*erfc(sqrt(3)*xi*sqrt(L**2))/(3*pi*eta*(L**2)**(5/2)) + sqrt(3)*L**4*erfc(2*sqrt(3)*xi*sqrt(L**2))/(6*pi*eta*(L**2)**(5/2)) + 4*sqrt(5)*L**4*erfc(sqrt(5)*xi*sqrt(L**2))/